In [17]:
import os
import numpy as np
import pandas as pd
import torch
import sagemaker
import torch
import torchtext

In [22]:
data_path1 = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/re/1000/raw/fold_0/raw_test_data_1000_30days_anony.csv'
data_path2 = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/re/1000/raw/fold_1/raw_test_data_1000_30days_anony.csv'
vocab_path1 = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/re/1000/raw/fold_0/vocab_1000_vall_30days'
vocab_path2 = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/re/1000/raw/fold_4/vocab_1000_vall_30days'

In [11]:
# df1 = pd.read_csv(data_path1, low_memory=False)
# df2 = pd.read_csv(data_path2, low_memory=False)
# print(df1.shape, df2.shape)
# df1.head()

In [12]:
# df = pd.concat([df1, df2], axis=0, ignore_index=True)
# print(df.shape)
# df.head()

In [18]:
torchtext.__version__

'0.8.0'

In [16]:
torch.__version__

'1.7.0+cu101'

In [19]:
!python --version

Python 3.6.11


In [23]:
#vocab1 = torch.load(vocab_path1)
vocab2 = torch.load(vocab_path2)

UnpicklingError: invalid load key, '9'.

In [8]:
anony_data_path = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/anonymize/RE/Data/Anonymized/365/readmission_input_targets_365_v2.csv'

In [4]:
df = pd.read_csv(anony_data_path, low_memory=False)
print(df.shape)

(1619158, 370)


In [6]:
df.tail()

,patient_id,discharge_dt,discharge_id,365,364,363,362,361,360,359,...,8,7,6,5,4,3,2,1,0,unplanned_readmission
1619153,4VL1HQUXY,20090715,4VL1HQUXY_20090715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"admission, d_20500, d_2866, d_51881, death, di...",False
1619154,ATIIMHYWM,20090209,ATIIMHYWM_20090209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"admission, d_2720, d_56211, d_71690, d_78650, ...","d_71690, d_78650, d_78659, discharge, h_78465,...",False
1619155,OC86YBLC1,20090327,OC86YBLC1_20090327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"admission, d_25000, d_4111, d_41401",NaN,discharge,False
1619156,9QGEDFGBM,20110124,9QGEDFGBM_20110124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"admission, d_40390, d_5589, d_78701",NaN,NaN,discharge,False
1619157,0NH0CGNC2,20100920,0NH0CGNC2_20100920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"admission, d_41090, d_41189, d_78650, d_78650,...",False


In [ ]:
job_name = tuner.latest_tuning_job.name
my_tuner = sagemaker.HyperparameterTuningJobAnalytics(job_name)
df = my_tuner.dataframe()
df.shape #4x18

{'HyperParameterTuningJobName': 'sagemaker-xgboost-201120-2310', 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:293790038521:hyper-parameter-tuning-job/sagemaker-xgboost-201120-2310', 'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian', 'HyperParameterTuningJobObjective': {'Type': 'Maximize', 'MetricName': 'validation:auc'}, 'ResourceLimits': {'MaxNumberOfTrainingJobs': 4, 'MaxParallelTrainingJobs': 4}, 'ParameterRanges': {'IntegerParameterRanges': [{'Name': 'max_depth', 'MinValue': '1', 'MaxValue': '10', 'ScalingType': 'Auto'}, {'Name': 'num_round', 'MinValue': '200', 'MaxValue': '500', 'ScalingType': 'Auto'}, {'Name': 'max_delta_step', 'MinValue': '0', 'MaxValue': '10', 'ScalingType': 'Auto'}], 'ContinuousParameterRanges': [{'Name': 'eta', 'MinValue': '0.1', 'MaxValue': '0.5', 'ScalingType': 'Auto'}, {'Name': 'alpha', 'MinValue': '0', 'MaxValue': '2', 'ScalingType': 'Auto'}, {'Name': 'gamma', 'MinValue': '0', 'MaxValue': '5', 'ScalingType': 'Auto'}, {'Name': 'colsample_bylevel', 'MinValue': '0.1', 'MaxValue': '1.0', 'ScalingType': 'Auto'}, {'Name': 'colsample_bynode', 'MinValue': '0.1', 'MaxValue': '1.0', 'ScalingType': 'Auto'}, {'Name': 'colsample_bytree', 'MinValue': '0.5', 'MaxValue': '1.0', 'ScalingType': 'Auto'}, {'Name': 'lambda', 'MinValue': '0', 'MaxValue': '1000', 'ScalingType': 'Auto'}, {'Name': 'min_child_weight', 'MinValue': '0', 'MaxValue': '120', 'ScalingType': 'Auto'}, {'Name': 'subsample', 'MinValue': '0.5', 'MaxValue': '1.0', 'ScalingType': 'Auto'}], 'CategoricalParameterRanges': []}, 'TrainingJobEarlyStoppingType': 'Off'}, 'TrainingJobDefinition': {'StaticHyperParameters': {'_tuning_objective_metric': 'validation:auc', 'eval_metric': 'auc', 'objective': 'binary:logistic', 'rate_drop': '0.3', 'scale_pos_weight': '4.0'}, 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-1', 'TrainingInputMode': 'File', 'MetricDefinitions': [{'Name': 'train:mae', 'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:aucpr', 'Regex': '.*\\[[0-9]+\\].*#011validation-aucpr:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:merror', 'Regex': '.*\\[[0-9]+\\].*#011train-merror:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:gamma-nloglik', 'Regex': '.*\\[[0-9]+\\].*#011train-gamma-nloglik:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:mae', 'Regex': '.*\\[[0-9]+\\].*#011validation-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:logloss', 'Regex': '.*\\[[0-9]+\\].*#011validation-logloss:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:mlogloss', 'Regex': '.*\\[[0-9]+\\].*#011train-mlogloss:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:cox-nloglik', 'Regex': '.*\\[[0-9]+\\].*#011train-cox-nloglik:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:f1', 'Regex': '.*\\[[0-9]+\\].*#011validation-f1:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:cox-nloglik', 'Regex': '.*\\[[0-9]+\\].*#011train-cox-nloglik:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:accuracy', 'Regex': '.*\\[[0-9]+\\].*#011train-accuracy:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:mse', 'Regex': '.*\\[[0-9]+\\].*#011train-mse:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:poisson-nloglik', 'Regex': '.*\\[[0-9]+\\].*#011validation-poisson-nloglik:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:tweedie-nloglik', 'Regex': '.*\\[[0-9]+\\].*#011train-tweedie-nloglik:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:error', 'Regex': '.*\\[[0-9]+\\].*#011train-error:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:ndcg', 'Regex': '.*\\[[0-9]+\\].*#011train-ndcg:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:map', 'Regex': '.*\\[[0-9]+\\].*#011validation-map:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:auc', 'Regex': '.*\\[[0-9]+\\].*#011validation-auc:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:gamma-deviance', 'Regex': '.*\\[[0-9]+\\].*#011validation-gamma-deviance:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:auc', 'Regex': '.*\\[[0-9]+\\].*#011train-auc:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:error', 'Regex': '.*\\[[0-9]+\\].*#011validation-error:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:merror', 'Regex': '.*\\[[0-9]+\\].*#011validation-merror:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:poisson-nloglik', 'Regex': '.*\\[[0-9]+\\].*#011train-poisson-nloglik:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:rmse', 'Regex': '.*\\[[0-9]+\\].*#011train-rmse:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:logloss', 'Regex': '.*\\[[0-9]+\\].*#011train-logloss:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:accuracy', 'Regex': '.*\\[[0-9]+\\].*#011validation-accuracy:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:aucpr', 'Regex': '.*\\[[0-9]+\\].*#011train-aucpr:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:tweedie-nloglik', 'Regex': '.*\\[[0-9]+\\].*#011validation-tweedie-nloglik:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:rmse', 'Regex': '.*\\[[0-9]+\\].*#011validation-rmse:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:gamma-deviance', 'Regex': '.*\\[[0-9]+\\].*#011train-gamma-deviance:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:mse', 'Regex': '.*\\[[0-9]+\\].*#011validation-mse:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:ndcg', 'Regex': '.*\\[[0-9]+\\].*#011validation-ndcg:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:f1', 'Regex': '.*\\[[0-9]+\\].*#011train-f1:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:mlogloss', 'Regex': '.*\\[[0-9]+\\].*#011validation-mlogloss:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'train:map', 'Regex': '.*\\[[0-9]+\\].*#011train-map:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'validation:gamma-nloglik', 'Regex': '.*\\[[0-9]+\\].*#011validation-gamma-nloglik:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}, {'Name': 'ObjectiveMetric', 'Regex': '.*\\[[0-9]+\\].*#011validation-auc:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'}]}, 'RoleArn': 'arn:aws:iam::293790038521:role/mrlpoc-sagemaker-role', 'InputDataConfig': [{'ChannelName': 'train', 'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/re/final-global/data/fold_0/100/unplanned_readmission/train', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'csv'}, {'ChannelName': 'validation', 'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/re/final-global/data/fold_0/100/unplanned_readmission/val', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'csv'}], 'OutputDataConfig': {'S3OutputPath': 's3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/re/final-global/xgboost/fold_0/2020-11-20-23-09-58/100/unplanned_readmission/output'}, 'ResourceConfig': {'InstanceType': 'ml.m4.16xlarge', 'InstanceCount': 2, 'VolumeSizeInGB': 30}, 'StoppingCondition': {'MaxRuntimeInSeconds': 86400}, 'EnableNetworkIsolation': False, 'EnableInterContainerTrafficEncryption': False, 'EnableManagedSpotTraining': False}, 'HyperParameterTuningJobStatus': 'Completed', 'CreationTime': datetime.datetime(2020, 11, 20, 23, 10, 2, 977000, tzinfo=tzlocal()), 'HyperParameterTuningEndTime': datetime.datetime(2020, 11, 20, 23, 30, 50, 953000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 11, 20, 23, 30, 50, 953000, tzinfo=tzlocal()), 'TrainingJobStatusCounters': {'Completed': 4, 'InProgress': 0, 'RetryableError': 0, 'NonRetryableError': 0, 'Stopped': 0}, 'ObjectiveStatusCounters': {'Succeeded': 4, 'Pending': 0, 'Failed': 0}, 'BestTrainingJob': {'TrainingJobName': 'sagemaker-xgboost-201120-2310-001-7d63ba25', 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:293790038521:training-job/sagemaker-xgboost-201120-2310-001-7d63ba25', 'CreationTime': datetime.datetime(2020, 11, 20, 23, 10, 13, tzinfo=tzlocal()), 'TrainingStartTime': datetime.datetime(2020, 11, 20, 23, 12, 41, tzinfo=tzlocal()), 'TrainingEndTime': datetime.datetime(2020, 11, 20, 23, 22, 56, tzinfo=tzlocal()), 'TrainingJobStatus': 'Completed', 'TunedHyperParameters': {'alpha': '0.3506179938150109', 'colsample_bylevel': '0.831342554812431', 'colsample_bynode': '0.47593142571804437', 'colsample_bytree': '0.5126519245758329', 'eta': '0.46287320167861645', 'gamma': '1.4073433566787807', 'lambda': '254.82578210226814', 'max_delta_step': '0', 'max_depth': '5', 'min_child_weight': '76.45545515753648', 'num_round': '461', 'subsample': '0.6143439303307552'}, 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc', 'Value': 0.5016300082206726}, 'ObjectiveStatus': 'Succeeded'}, 'ResponseMetadata': {'RequestId': 'a6cb03c4-489c-465e-8cfd-31ec69cd8c6a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a6cb03c4-489c-465e-8cfd-31ec69cd8c6a', 'content-type': 'application/x-amz-json-1.1', 'content-length': '8608', 'date': 'Mon, 23 Nov 2020 17:05:16 GMT'}, 'RetryAttempts': 0}}

['DEFAULT_ESTIMATOR_CLS_NAME', 'DEFAULT_ESTIMATOR_MODULE', 'SAGEMAKER_ESTIMATOR_CLASS_NAME', 'SAGEMAKER_ESTIMATOR_MODULE', 'TUNING_JOB_NAME_MAX_LENGTH', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_estimator', '_attach_with_training_details', '_attach_with_training_details_list', '_create_warm_start_tuner', '_current_job_name', '_ensure_last_tuning_job', '_extract_hyperparameters_from_parameter_ranges', '_fit_with_estimator', '_fit_with_estimator_dict', '_get_best_training_job', '_hyperparameter_ranges', '_hyperparameter_ranges_dict', '_prepare_estimator', '_prepare_estimator_cls', '_prepare_estimator_for_tuning', '_prepare_estimator_from_job_description', '_prepare_for_tuning', '_prepare_init_params_from_job_description', '_prepare_job_name_for_tuning', '_prepare_parameter_ranges_for_tuning', '_prepare_parameter_ranges_from_job_description', '_prepare_static_hyperparameters', '_prepare_static_hyperparameters_for_tuning', '_validate_create_tuner_inputs', '_validate_dict_argument', '_validate_estimator_dict', '_validate_parameter_range', '_validate_parameter_ranges', 'analytics', 'attach', 'base_tuning_job_name', 'best_estimator', 'best_training_job', 'create', 'delete_endpoint', 'deploy', 'describe', 'early_stopping_type', 'estimator', 'estimator_dict', 'fit', 'hyperparameter_ranges', 'hyperparameter_ranges_dict', 'identical_dataset_and_algorithm_tuner', 'latest_tuning_job', 'max_jobs', 'max_parallel_jobs', 'metric_definitions', 'metric_definitions_dict', 'objective_metric_name', 'objective_metric_name_dict', 'objective_type', 'sagemaker_session', 'static_hyperparameters', 'static_hyperparameters_dict', 'stop_tuning_job', 'strategy', 'tags', 'transfer_learning_tuner', 'wait', 'warm_start_config'](Pdb) dir(tuner.analytics)

In [ ]:
old_version_train_path = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/readmissions_debug/train_1000.csv'
old_version_val_path = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/readmissions_debug/val_1000.csv'
new_version_train_path = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/readmissions_debug/raw_train_data_flatten.csv'
new_version_val_path = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/readmissions_debug/raw_test_data_flatten.csv'

In [ ]:
print('Reading old version...')
df_tr_old = pd.read_csv(old_version_train_path, low_memory=False)
df_val_old = pd.read_csv(old_version_val_path, low_memory=False)
print('Aggregating old version...')
df = pd.concat([df_tr_old, df_val_old], ignore_index=True, axis=1)
print(df_val_old.shape, df.shape)
output_dir = os.path.dirname(old_version_train_path)
output_path = os.path.join(output_dir, 'old.csv')
df.to_csv(output_path, index=False)

print('Reading new version...')
df_tr_new = pd.read_csv(new_version_train_path, low_memory=False)
df_val_new = pd.read_csv(new_version_val_path, low_memory=False)
print('Aggregating new version...')
df = pd.concat([df_tr_new, df_val_new], ignore_index=True, axis=1)
print(df_val_new.shape, df.shape)
output_dir = os.path.dirname(new_version_train_path)
output_path = os.path.join(output_dir, 'new.csv')
df.to_csv(output_path, index=False)

In [ ]:
print(df_val_old.shape)
df_val_old.head()

In [ ]:
print(df_val_new.shape)
df_val_new.head()

In [ ]:
data_path = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/re/1000/raw/fold_0/test/raw_test_data_flatten_30days.csv'
#vocab_path = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final/re/fold_0/vocab/dev_vocab_d30_s30_vpos'
vocab_path = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/re/1000/raw/fold_0/vocab/vocab_d30_s30_vpos'

In [ ]:
df = pd.read_csv(data_path)

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.tail()

In [ ]:
df['unplanned_readmission'].value_counts(normalize=True)

In [ ]:
vocab = torch.load(vocab_path)

In [ ]:
freq_vocab = vocab.freqs
del freq_vocab[0]
freq_vocab.most_common(10)

In [ ]:
def get_frequent_features(vocab, num_features, codes_only=True, exclusion_list=[]): 
    """Get the most frequent codes/features."""
    num_exc = len(exclusion_list) + 100
    features = vocab.freqs.most_common(num_features + num_exc)
    if codes_only:
        features = [word[0] for word in features if word[0] not in exclusion_list and ('_' in word[0])]
    else:
        features = [word[0] for word in features if word[0] not in exclusion_list]
    features = [word for word in features if 'day' not in word] #Exclude day features
    features = features[:num_features]
    return features

In [ ]:
num_features = 100
exclusion_list = ['nan', 'pad', 'unk']
codes_only = True

In [ ]:
features = get_frequent_features(vocab, num_features, codes_only, exclusion_list)

In [ ]:
print('Total features:', len(features))
features[:10]